In [1]:
import numpy as np
import pandas as pd
import patsy
from sklearn.ensemble import RandomForestClassifier 

In [2]:
df = pd.read_csv("data/train.csv") 
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


/Users/hqiu/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [3]:
df['Gender'] = df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

In [4]:
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Gender
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1


In [5]:
median_ages = np.zeros((2,3))

for i in range(0, 2):
    for j in range(0, 3):
        median_ages[i,j] = df[(df['Gender'] == i) & (df['Pclass'] == j+1)]['Age'].dropna().median()

median_ages

array([[ 35. ,  28. ,  21.5],
       [ 40. ,  30. ,  25. ]])

In [6]:
df['AgeFill'] = df['Age']

for i in range(0, 2):
    for j in range(0, 3):
        df.loc[ (df.Age.isnull()) & (df.Gender == i) & (df.Pclass == j+1), 'AgeFill'] = median_ages[i,j]

In [7]:
df[ df['Age'].isnull() ][['Gender','Pclass','Age','AgeFill']].head(10)

,Gender,Pclass,Age,AgeFill
5,1,3,NaN,25.0
17,1,2,NaN,30.0
19,0,3,NaN,21.5
26,1,3,NaN,25.0
28,0,3,NaN,21.5
29,1,3,NaN,25.0
31,0,1,NaN,35.0
32,0,3,NaN,21.5
36,1,3,NaN,25.0
42,1,3,NaN,25.0


In [8]:
df.loc[ (df.Cabin.notnull()), 'Cabin' ] = "Yes"
df.loc[ (df.Cabin.isnull()), 'Cabin' ] = "No"

dummies_Cabin = pd.get_dummies(df['Cabin'], prefix= 'Cabin')

dummies_Embarked = pd.get_dummies(df['Embarked'], prefix= 'Embarked')

dummies_Sex = pd.get_dummies(df['Sex'], prefix= 'Sex')

dummies_Pclass = pd.get_dummies(df['Pclass'], prefix= 'Pclass')

df = pd.concat([df, dummies_Cabin, dummies_Embarked, dummies_Sex, dummies_Pclass], axis=1)
df.drop(['Pclass', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis=1, inplace=True)

df.describe()

,PassengerId,Survived,SibSp,Parch,Fare,Gender,AgeFill,Cabin_No,Cabin_Yes,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,0.523008,0.381594,32.204208,0.647587,29.112424,0.771044,0.228956,0.188552,0.086420,0.722783,0.352413,0.647587,0.242424,0.206510,0.551066
std,257.353842,0.486592,1.102743,0.806057,49.693429,0.477990,13.304424,0.420397,0.420397,0.391372,0.281141,0.447876,0.477990,0.477990,0.428790,0.405028,0.497665
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,0.000000,0.000000,7.910400,0.000000,21.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,446.000000,0.000000,0.000000,0.000000,14.454200,1.000000,26.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,668.500000,1.000000,1.000000,0.000000,31.000000,1.000000,36.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000
max,891.000000,1.000000,8.000000,6.000000,512.329200,1.000000,80.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
df.head(5)

,PassengerId,Survived,SibSp,Parch,Fare,Gender,AgeFill,Cabin_No,Cabin_Yes,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
0,1,0,1,0,7.2500,1,22.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,2,1,1,0,71.2833,0,38.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,3,1,0,0,7.9250,0,26.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,4,1,1,0,53.1000,0,35.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
4,5,0,0,0,8.0500,1,35.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [39]:
from sklearn import linear_model

train_df = df.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.*')

train_df_m = train_df.as_matrix()

X = train_df_m[:, 1:]
y = train_df_m[:, 0]

lr = linear_model.LogisticRegression(C=1.0, penalty='l1', tol=1e-6)
lr.fit(X,y)
print lr

print lr.score(X,y)

from sklearn import cross_validation

print cross_validation.cross_val_score(lr, X, y, cv=5)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=1e-06,
          verbose=0, warm_start=False)
0.804713804714
[ 0.79888268  0.79888268  0.80337079  0.78089888  0.81920904]


In [17]:
df_predict = pd.read_csv("data/test.csv")
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [18]:
df_predict['Gender'] = df_predict['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

In [19]:
df_predict['AgeFill'] = df_predict['Age']

for i in range(0, 2):
    for j in range(0, 3):
        df_predict.loc[ (df_predict.Age.isnull()) & (df_predict.Gender == i) & (df_predict.Pclass == j+1), \
                       'AgeFill'] = median_ages[i,j]

In [20]:
print df_predict['Fare'].dropna().median()

14.4542


In [21]:
print df_predict[df_predict.Fare.isnull()]

     PassengerId  Pclass                Name   Sex   Age  SibSp  Parch Ticket  \
152         1044       3  Storey, Mr. Thomas  male  60.5      0      0   3701   

     Fare Cabin Embarked  Gender  AgeFill  
152   NaN   NaN        S       1     60.5  


In [23]:
# df_predict.loc[ df_predict['Fare'].isnull(), 'Fare' ] = df_predict['Fare'].dropna().median()
# print df_predict[df_predict.PassengerId == 1044]

if len(df_predict.Fare[ df_predict.Fare.isnull() ]) > 0:
    median_fare = np.zeros(3)
    for f in range(0,3):                                              # loop 0 to 2
        median_fare[f] = df_predict[ df_predict.Pclass == f+1 ]['Fare'].dropna().median()
    for f in range(0,3):                                              # loop 0 to 2
        df_predict.loc[ (df_predict.Fare.isnull()) & (df_predict.Pclass == f+1 ), 'Fare'] = median_fare[f]

In [24]:
df_predict.loc[ (df_predict.Cabin.notnull()), 'Cabin' ] = "Yes"
df_predict.loc[ (df_predict.Cabin.isnull()), 'Cabin' ] = "No"

dummies_Cabin = pd.get_dummies(df_predict['Cabin'], prefix= 'Cabin')

dummies_Embarked = pd.get_dummies(df_predict['Embarked'], prefix= 'Embarked')

dummies_Sex = pd.get_dummies(df_predict['Sex'], prefix= 'Sex')

dummies_Pclass = pd.get_dummies(df_predict['Pclass'], prefix= 'Pclass')

df_predict = pd.concat([df_predict, dummies_Cabin, dummies_Embarked, dummies_Sex, dummies_Pclass], axis=1)
df_predict.drop(['Pclass', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], axis=1, inplace=True)

df_predict.describe()

,PassengerId,SibSp,Parch,Fare,Gender,AgeFill,Cabin_No,Cabin_Yes,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,0.447368,0.392344,35.560845,0.636364,29.357656,0.782297,0.217703,0.244019,0.110048,0.645933,0.363636,0.636364,0.255981,0.222488,0.521531
std,120.810458,0.896760,0.981429,55.856972,0.481622,12.957412,0.413179,0.413179,0.430019,0.313324,0.478803,0.481622,0.481622,0.436934,0.416416,0.500135
min,892.000000,0.000000,0.000000,0.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,996.250000,0.000000,0.000000,7.895800,0.000000,21.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1100.500000,0.000000,0.000000,14.454200,1.000000,25.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,1204.750000,1.000000,0.000000,31.471875,1.000000,36.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000
max,1309.000000,8.000000,9.000000,512.329200,1.000000,76.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [25]:
df_predict.head(5)

,PassengerId,SibSp,Parch,Fare,Gender,AgeFill,Cabin_No,Cabin_Yes,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
0,892,0,0,7.8292,1,34.5,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,893,1,0,7.0000,0,47.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2,894,0,0,9.6875,1,62.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,895,0,0,8.6625,1,27.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,896,1,1,12.2875,0,22.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


In [27]:
test_data = df_predict.filter(regex='Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.*')
y_predict = lr.predict(test_data)

In [29]:
from pandas import DataFrame

result = pd.DataFrame({'PassengerId':df_predict['PassengerId'].as_matrix(), 'Survived':y_predict.astype(np.int32)})
result.to_csv("data/output/logit_regression.csv", index=False)

In [37]:
pd.DataFrame({"columns":list(train_df.columns)[1:], "coef":list(lr.coef_.T)})

,coef,columns
0,[-0.200579320968],SibSp
1,[-0.056174677745],Parch
2,[0.0],Cabin_No
3,[0.944512972905],Cabin_Yes
4,[0.200448115894],Embarked_C
5,[0.0],Embarked_Q
6,[-0.341858436057],Embarked_S
7,[1.77306436146],Sex_female
8,[-0.889475760378],Sex_male
9,[0.0],Pclass_1
